<div class="alert alert-block alert-info">
<p style="font-size:24px;text-align:center"><b>Experimental Comparison (Feb 2022 Data Set)</b>
</div>

In [ ]:
import numpy as np

In [ ]:
# Laser pulse
num_laser_slices = 20
pulse_ncells = 128

# Crystal
num_crystal_slices = 10
radial_n2 = 1

# Drift
L_drift = 0.5 #[m]

In [ ]:
colorbar_min = 1397.3978718313067 #1859.2289027463828
colorbar_max = 656068371.7706386 #871345988.9991088

colorbar_min2 = -6669381.44800365 #-8873569.171015844
colorbar_max2 = 92993576.55883849 #123727370.53688169

### Imports, Files, Crystal/Laser Params

In [ ]:
import sys, copy
from pykern.pkcollections import PKDict
import os

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import crystal
from rslaser.optics import lens
import rslaser.utils.srwl_uti_data as srwutil
import uti_plot

import srwlib

import scipy.constants as const
from scipy import interpolate
from scipy import special

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['pcolor.shading'] ='auto'
%matplotlib inline

In [ ]:
package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data/20220218')
ccd_name = 'photon_count_pump_off.txt'
wfs_name = 'phase_pump_off.txt'

files = PKDict(
    meta = os.path.join(package_data_dir, 'meta_data.dat'),
    ccd = os.path.join(package_data_dir, ccd_name),
    wfs = os.path.join(package_data_dir, wfs_name)
)

z       = np.array([ 0.0,  0.5,  1.0, 1.5, 2.0, 2.5])/100.0
n2_plot = np.array([30.5, 18.3, 10.4, 5.9, 3.3, 1.9])
plot_fit = interpolate.splrep(z, n2_plot)

In [ ]:
w0 = 1.64e-3

params = PKDict(
    photon_e_ev = 1.5498, # Photon energy [eV], calculated from 800nm wavelength
    nslice      = num_laser_slices,
    pulseE      = 1.0e-6,
    tau_fwhm    = 300.0e-12 / np.sqrt(2.),
    tau_0    = 300.0e-12 / np.sqrt(2.),
    sigx_waist  = w0,
    sigy_waist  = w0,
    nx_slice    = pulse_ncells,
)

L_cryst = 0.025
new_z = (L_cryst /num_crystal_slices) *(np.arange(num_crystal_slices)+0.5)

crystal_params = PKDict(
    length      = L_cryst, # [m]
    nslice      = num_crystal_slices,
    n0          = [1.76 for _ in range(num_crystal_slices)],
    n2          = interpolate.splev(new_z, plot_fit).tolist(),
    pop_inversion_n_cells=128,
    pop_inversion_mesh_extent=0.01,  # [m]
    pop_inversion_crystal_alpha=120.0,  # [1/m], 1.2 1/cm
    pop_inversion_pump_waist=w0,  # [m]
    pop_inversion_pump_wavelength=532.0e-9,  # [m]
    pop_inversion_pump_energy=0.0211,  # [J], pump laser energy onto the crystal
    pop_inversion_pump_type='left',
    pop_inversion_pump_gaussian_order=2.0,
    pop_inversion_pump_offset_x=0.0e-3,
    pop_inversion_pump_offset_y=0.0e-3,
)

cell_dx = (2.0 *crystal_params.pop_inversion_mesh_extent)/crystal_params.pop_inversion_n_cells
cell_volume = cell_dx**2.0 *(crystal_params.length /crystal_params.nslice)

In [ ]:
def plot_excited_states(crystal):
    long_excited_states, trans_excited_states = crystal.extract_excited_states()
    z = np.linspace(0,2.5,len(long_excited_states))
    x = np.linspace(-crystal.params.pop_inversion_mesh_extent,
                    crystal.params.pop_inversion_mesh_extent,
                    crystal.params.pop_inversion_n_cells)
    
    plt.figure()
    plt.plot(z,long_excited_states, 'k')
    plt.xlabel('Longitudinal Distance [cm]')
    plt.ylabel('Num Excited States')
    plt.show()
    
    fig = plt.figure()
    ax = fig.gca()
    plt.pcolormesh(x*(1e3), x*(1e3), trans_excited_states, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Total Num Excited States')
    plt.show()

    uti_plot.uti_plot2d1d(
        trans_excited_states.flatten(),
        [-crystal.params.pop_inversion_mesh_extent, crystal.params.pop_inversion_mesh_extent, crystal.params.pop_inversion_n_cells],
        [-crystal.params.pop_inversion_mesh_extent, crystal.params.pop_inversion_mesh_extent, crystal.params.pop_inversion_n_cells],
        0,
        0,
        ['Horizontal Position', 'Vertical Position', 'Total Num Excited States'],
        ['m', 'm', 'num'],
        True)
    
    print(np.sum(long_excited_states),np.sum(trans_excited_states))

***
## Initial Intensity and Phase

In [ ]:
# Initialize laser pulse
LP_temp = pulse.LaserPulse(params, files)
LP_temp.zero_phase()

# Plot wavefront
lp_wfr_x = np.linspace(LP_temp.slice[0].wfr.mesh.xStart,LP_temp.slice[0].wfr.mesh.xFin,LP_temp.slice[0].wfr.mesh.nx)
lp_wfr_y = np.linspace(LP_temp.slice[0].wfr.mesh.yStart,LP_temp.slice[0].wfr.mesh.yFin,LP_temp.slice[0].wfr.mesh.ny)

In [ ]:
e_total_t = LP_temp.extract_total_2d_elec_fields()
intensity_2D_t = 0.5 *const.c *const.epsilon_0 *(e_total_t.re**2.0 + e_total_t.im**2.0)

intens0 = intensity_2D_t.flatten()
wfr0 = LP_temp.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (initial)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(lp_wfr_x*(1e3), lp_wfr_y*(1e3), intensity_2D_t, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (initial)')

In [ ]:
phase_t = LP_temp.extract_total_2d_phase()

phase0 = phase_t.flatten()
wfr0 = LP_temp.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (initial)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(lp_wfr_x*(1e3), lp_wfr_y*(1e3), phase_t, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (initial)')

In [ ]:
LP_temp.calc_total_energy()

# Propagate with drift and gain

In [ ]:
### Initialize laser pulse
thisPulse_dg = pulse.LaserPulse(params, files)
thisPulse_dg.zero_phase()
e_crystal = crystal.Crystal(crystal_params)
plot_excited_states(e_crystal)

print('\nPropagating ', thisPulse_dg.nslice, 'laser slices through ', e_crystal.nslice,' crystal slices')
thisPulse_dg = e_crystal.propagate(thisPulse_dg, 'n0n2_srw', 1, radial_n2)
plot_excited_states(e_crystal)

# instantiate a drift
e_drift = lens.Drift_srw(L_drift)
thisPulse_dg = e_drift.propagate(thisPulse_dg, 'default')

wfr_dg=thisPulse_dg.slice[0].wfr
x_dg=np.linspace(wfr_dg.mesh.xStart,wfr_dg.mesh.xFin,wfr_dg.mesh.nx)
y_dg=np.linspace(wfr_dg.mesh.yStart,wfr_dg.mesh.yFin,wfr_dg.mesh.ny)

In [ ]:
e_total_dg = thisPulse_dg.extract_total_2d_elec_fields()
intensity_2D_dg = 0.5 *const.c *const.epsilon_0 *(e_total_dg.re**2.0 + e_total_dg.im**2.0)

intens0 = intensity_2D_dg.flatten()
wfr0 = thisPulse_dg.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (propagation with gain)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_dg*(1e3), y_dg*(1e3), intensity_2D_dg, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (propagation with gain)')


In [ ]:
phase_dg = thisPulse_dg.extract_total_2d_phase()

phase0 = phase_dg.flatten()
wfr0 = thisPulse_dg.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (propagation with gain)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_dg*(1e3), y_dg*(1e3), phase_dg, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (propagation with gain)')

# Propagate with drift and without gain

In [ ]:
# Initialize laser pulse
thisPulse_d = pulse.LaserPulse(params, files)
thisPulse_d.zero_phase()
e_crystal = crystal.Crystal(crystal_params)
plot_excited_states(e_crystal)

print('\nPropagating ', thisPulse_d.nslice, 'laser slices through ', e_crystal.nslice,' crystal slices')
thisPulse_d = e_crystal.propagate(thisPulse_d, 'n0n2_srw', 0, radial_n2)
plot_excited_states(e_crystal)

# instantiate a drift
e_drift = lens.Drift_srw(L_drift)
thisPulse_d = e_drift.propagate(thisPulse_d, 'default')

wfr_d=thisPulse_d.slice[0].wfr
x_d=np.linspace(wfr_d.mesh.xStart,wfr_d.mesh.xFin,wfr_d.mesh.nx)
y_d=np.linspace(wfr_d.mesh.yStart,wfr_d.mesh.yFin,wfr_d.mesh.ny)

In [ ]:
e_total_d = thisPulse_d.extract_total_2d_elec_fields()
intensity_2D_d = 0.5 *const.c *const.epsilon_0 *(e_total_d.re**2.0 + e_total_d.im**2.0)

intens0 = intensity_2D_d.flatten()
wfr0 = thisPulse_d.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (propagation without gain)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_d*(1e3), y_d*(1e3), intensity_2D_d, cmap=plt.cm.viridis, shading='auto')#, vmin=colorbar_min, vmax=colorbar_max)
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (simulated focusing)')

In [ ]:
phase_d = thisPulse_d.extract_total_2d_phase()

phase0 = phase_d.flatten()
wfr0 = thisPulse_d.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (propagation without gain)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_d*(1e3), y_d*(1e3), phase_d, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (propagation, no gain)')

# Experiment: Off-peak Timing

In [ ]:
files_op = PKDict(
    meta = os.path.join(package_data_dir, 'meta_data.dat'),
    ccd = os.path.join(package_data_dir, 'photon_count_off_peak_timing.txt'), 
    wfs = os.path.join(package_data_dir, 'phase_off_peak_timing.txt'),
)

offpeakPulse = pulse.LaserPulse(params, files_op)
wfr_exp_final_op = offpeakPulse.slice[0].wfr

# _op = _off_peak

x_exp_final_op=np.linspace(wfr_exp_final_op.mesh.xStart,wfr_exp_final_op.mesh.xFin,wfr_exp_final_op.mesh.nx)
y_exp_final_op=np.linspace(wfr_exp_final_op.mesh.yStart,wfr_exp_final_op.mesh.yFin,wfr_exp_final_op.mesh.ny)

In [ ]:
e_total_op = offpeakPulse.extract_total_2d_elec_fields()
intensity_2D_exp_final_op = 0.5 *const.c *const.epsilon_0 *(e_total_op.re**2.0 + e_total_op.im**2.0)

intens0 = intensity_2D_exp_final_op.flatten()
wfr0 = offpeakPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (off-peak timing)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(y_exp_final_op*(1e3), x_exp_final_op*(1e3), intensity_2D_exp_final_op, cmap=plt.cm.viridis, shading='auto', vmin=colorbar_min, vmax=colorbar_max)
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (experimental focusing)')

In [ ]:
phase_exp_final_op = offpeakPulse.extract_total_2d_phase()

phase0 = phase_exp_final_op.flatten()
wfr0 = offpeakPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (off-peak timing)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_exp_final_op*(1e3), y_exp_final_op*(1e3), phase_exp_final_op, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (off-peak timing)')

# Experiment: Peak Timing

In [ ]:
files_p = PKDict(
    meta = os.path.join(package_data_dir, 'meta_data.dat'),
    ccd = os.path.join(package_data_dir, 'photon_count_peak_timing.txt'),
    wfs = os.path.join(package_data_dir, 'phase_peak_timing.txt'),
)

peakPulse = pulse.LaserPulse(params, files_p)
wfr_exp_final = peakPulse.slice[0].wfr

x_exp_final=np.linspace(wfr_exp_final.mesh.xStart,wfr_exp_final.mesh.xFin,wfr_exp_final.mesh.nx)
y_exp_final=np.linspace(wfr_exp_final.mesh.yStart,wfr_exp_final.mesh.yFin,wfr_exp_final.mesh.ny)

In [ ]:
e_total_p = peakPulse.extract_total_2d_elec_fields()
intensity_2D_exp_final = 0.5 *const.c *const.epsilon_0 *(e_total_p.re**2.0 + e_total_p.im**2.0)

intens0 = intensity_2D_exp_final.flatten()
wfr0 = peakPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (peak timing)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(y_exp_final*(1e3), x_exp_final*(1e3), intensity_2D_exp_final, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (peak timing)')

In [ ]:
phase_2d_exp_final = peakPulse.extract_total_2d_phase()

phase0 = phase_2d_exp_final.flatten()
wfr0 = peakPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (peak timing)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(y_exp_final*(1e3), x_exp_final*(1e3), phase_2d_exp_final, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (peak timing)')

# Comparison: Intensity Subtractions

In [ ]:
diff_intensity_2D_exp_final = intensity_2D_exp_final - intensity_2D_exp_final_op
intens0 = diff_intensity_2D_exp_final.flatten()
wfr0 = peakPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (peak timing minus off-peak timing)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_exp_final*(1e3), y_exp_final*(1e3), diff_intensity_2D_exp_final, cmap=plt.cm.viridis, shading='auto', vmin=colorbar_min2, vmax=colorbar_max2)
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (experimental gain)')

In [ ]:
diff_intensity_2D_sim = intensity_2D_dg - intensity_2D_d
intens0 = diff_intensity_2D_sim.flatten()
wfr0 = thisPulse_dg.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (with gain minus without gain)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_dg*(1e3), y_dg*(1e3), diff_intensity_2D_sim, cmap=plt.cm.viridis, shading='auto',vmin=colorbar_min2, vmax=colorbar_max2)
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (simulated gain)')

# Comparison: Phase Lineouts

In [ ]:
compare = phase_exp_final_op[:,int(len(y_exp_final_op)/2.0)]
sim_phase = phase_d[:,int(len(y_d)/2.0)]

scale_factor = np.max(compare) -np.max(sim_phase)
fig = plt.figure()
ax = fig.gca()
plt.plot(x_exp_final_op*(1e3), compare, 'r', label='Experiment')
plt.plot(x_d*(1e3), scale_factor +sim_phase, '--k', label='Simulation +{:.3}'.format(scale_factor))
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
ax.set_ylabel(r'Phase Lineout')
ax.set_xlabel(r'Vertical Position [mm]')
ax.set_title('Phase: Focusing')

compare = phase_exp_final_op[int(len(x_exp_final_op)/2.0),:]
sim_phase = phase_d[int(len(x_d)/2.0),:]

scale_factor = np.max(compare) -np.max(sim_phase)
fig = plt.figure()
ax = fig.gca()
plt.plot(y_exp_final_op*(1e3), compare, 'r', label='Experiment')
plt.plot(y_d*(1e3), scale_factor +sim_phase, '--k', label='Simulation +{:.3}'.format(scale_factor))
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
ax.set_ylabel(r'Phase Lineout')
ax.set_xlabel(r'Horizontal Position [mm]')
ax.set_title('Phase: Focusing')

In [ ]:
print(np.min(intensity_2D_d),np.max(intensity_2D_d))
print(np.min(intensity_2D_exp_final_op),np.max(intensity_2D_exp_final_op))
print(np.min(diff_intensity_2D_exp_final),np.max(diff_intensity_2D_exp_final))
print(np.min(diff_intensity_2D_sim),np.max(diff_intensity_2D_sim))

In [ ]:
sim_gain = np.max(diff_intensity_2D_sim)
exp_gain = np.max(diff_intensity_2D_exp_final)

gain_pd = 100.0 *np.abs(sim_gain - exp_gain) / ((sim_gain + exp_gain)/2.0)
print('% Difference Gain: ',gain_pd)

sim_focus = np.max(intensity_2D_d)
exp_focus = np.max(intensity_2D_exp_final_op)

focus_pd = 100.0 *np.abs(sim_focus - exp_focus) / ((sim_focus + exp_focus)/2.0)
print('% Difference Focusing: ',focus_pd)